## Cadenas de Markov 

Para diseñar un modelo de aprendizaje por refuerzo utilizando Cadenas de Markov para predecir el ganador de la Champions League, primero necesitamos entender y procesar los datos que has subido. Vamos a comenzar cargando y explorando tus datos para ver cómo están estructurados y qué tipo de información contienen

In [2]:
import pandas as pd

# Cargar el archivo CSV para revisar su contenido
data_path = '../../data/partidos_limpio.csv'
data = pd.read_csv(data_path)

# Mostrar las primeras filas del dataframe
data.head()

,Season,Round,Day,Date,Home,Score,Away,Venue,Referee
0,2022-2023,Round of 16,Tue,2023-02-14,Milan,1-0,Tottenham,Stadio Giuseppe Meazza,Sandro Schärer
1,2022-2023,Round of 16,Tue,2023-02-14,Paris S-G,0-1,Bayern Munich,Parc des Princes,Michael Oliver
2,2022-2023,Round of 16,Wed,2023-02-15,Club Brugge,0-2,Benfica,Jan Breydelstadion,Davide Massa
3,2022-2023,Round of 16,Wed,2023-02-15,Dortmund,1-0,Chelsea,Signal Iduna Park,Jesús Gil
4,2022-2023,Round of 16,Tue,2023-02-21,Liverpool,2-5,Real Madrid,Anfield,István Kovács


Para un modelo básico de Cadena de Markov, podríamos definir los estados como los posibles resultados de un partido (victoria del equipo local, empate, victoria del equipo visitante) y las transiciones basadas en la probabilidad de pasar de un resultado a otro en partidos consecutivos. Sin embargo, un enfoque más complejo y potencialmente más útil podría considerar aspectos como la fuerza relativa de los equipos, las ubicaciones de los partidos (casa o fuera), entre otros.

#### 1. Paso 1

 Primero, necesitamos extraer el resultado de cada partido en términos de victoria del equipo local, empate o victoria del equipo visitante. Esto implica:

* Separar los goles de cada equipo del campo 'Score'.
* Comparar los goles para determinar el resultado del partido (victoria local, empate, victoria visitante).

In [3]:
# Separar los goles de cada equipo
data[['Home_goals', 'Away_goals']] = data['Score'].str.split('-', expand=True).astype(int)

# Determinar el resultado del partido: 0 = empate, 1 = victoria local, 2 = victoria visitante
data['Result'] = (data['Home_goals'] > data['Away_goals']).astype(int) + (data['Away_goals'] > data['Home_goals']).astype(int) * 2

# Revisar los datos procesados
data[['Home', 'Away', 'Home_goals', 'Away_goals', 'Result']].head()

,Home,Away,Home_goals,Away_goals,Result
0,Milan,Tottenham,1,0,1
1,Paris S-G,Bayern Munich,0,1,2
2,Club Brugge,Benfica,0,2,2
3,Dortmund,Chelsea,1,0,1
4,Liverpool,Real Madrid,2,5,2


Una vez hecho esto, podremos utilizar estos resultados para definir los estados de nuestra Cadena de Markov. Vamos a procesar estos datos ahora.

He procesado los resultados de los partidos para determinar si el equipo local ganó (representado por 1), si el partido resultó en empate (representado por 0), o si ganó el equipo visitante (representado por 2). Con esta información, ya podemos avanzar hacia la definición y construcción de la matriz de transición para la Cadena de Markov.

#### 2. Definición de la matriz de transición

Para cada estado (victoria local, empate, victoria visitante), necesitamos calcular la probabilidad de transición hacia cada uno de los otros estados. Esto implica:

* Contar cuántas veces cada resultado (estado) sigue a otro en partidos consecutivos.
* Dividir cada conteo por el total de partidos que terminaron en el estado inicial para obtener la probabilidad de transición.

In [4]:
import numpy as np

# Inicializar la matriz de transición
transition_matrix = np.zeros((3, 3))

# Calcular las transiciones de estados
for i in range(3):
    # Filtrar partidos donde el resultado anterior es 'i'
    subset = data[data['Result'].shift(1) == i]
    
    # Contar transiciones hacia otros estados
    for j in range(3):
        transition_matrix[i, j] = (subset['Result'] == j).sum()

# Normalizar las filas para obtener probabilidades
transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)

transition_matrix

array([[0.224     , 0.512     , 0.264     ],
       [0.20212766, 0.5177305 , 0.28014184],
       [0.24096386, 0.42771084, 0.3313253 ]])

He calculado la matriz de transición para nuestro modelo de Cadena de Markov. Cada fila representa el estado inicial (victoria local, empate, victoria visitante), y cada columna representa el estado siguiente. Los valores en la matriz son las probabilidades de transición entre estos estados:

* La primera fila corresponde a los partidos que terminaron con una victoria local. Las probabilidades de que el siguiente partido termine en victoria local, empate, y victoria visitante son aproximadamente 22.4%, 51.2%, y 26.4% respectivamente.

* La segunda fila para los empates muestra probabilidades de 20.2% para victoria local, 51.8% para otro empate, y 28.0% para victoria visitante en el siguiente partido.

* La tercera fila, para victorias visitantes, tiene probabilidades de 24.1% para victoria local, 42.8% para empate, y 33.1% para otra victoria visitante en el partido siguiente.

#### 3. Simulación

Una vez que tenemos la matriz de transición definida, vamos a utilizar esta y los datos que tenemos de esta temporada para determinar el ganador de la Champions League 2023-2024

In [6]:
# Cargar el archivo CSV de la temporada 2023-2024 para revisar su contenido
data_path_new = '../../data/partidos_2023-2024_limpio.csv'
data_new = pd.read_csv(data_path_new)

# Mostrar las primeras filas del nuevo dataframe limpio
data_new.head()

,Season,Round,Day,Date,Home,Score,Away,Venue,Referee
0,2023-2024,Round of 16,Tue,2024-02-13,RB Leipzig,0-1,Real Madrid,Red Bull Arena,Irfan Peljto
1,2023-2024,Round of 16,Tue,2024-02-13,FC Copenhagen,1-3,Manchester City,Parken,José Sánchez
2,2023-2024,Round of 16,Wed,2024-02-14,Paris S-G,2-0,Real Sociedad,Parc des Princes,Marco Guida
3,2023-2024,Round of 16,Wed,2024-02-14,Lazio,1-0,Bayern Munich,Stadio Olimpico,François Letexier
4,2023-2024,Round of 16,Tue,2024-02-20,PSV Eindhoven,1-1,Dortmund,Philips Stadion,Srđan Jovanović


In [ ]:
# Filtrar los datos para excluir partidos sin resultados registrados (donde 'Score' es NaN)
played_data = data_new.dropna(subset=['Score'])

# Extraer los goles de cada equipo de los partidos jugados
played_data['Home_goals'] = played_data['Score'].str.extract('(\d+)-')[0].astype(int)
played_data['Away_goals'] = played_data['Score'].str.extract('-(\d+)')[0].astype(int)

# Determinar el resultado del partido: 0 = empate, 1 = victoria local, 2 = victoria visitante
played_data['Result'] = (played_data['Home_goals'] > played_data['Away_goals']).astype(int) + \
                        (played_data['Away_goals'] > played_data['Home_goals']).astype(int) * 2

# Identificar partidos sin resultados para simulación
unplayed_matches = data_new[data_new['Score'].isna()]

played_data[['Home', 'Away', 'Home_goals', 'Away_goals', 'Result']], unplayed_matches[['Home', 'Away']]


In [5]:
# Cargar los datos de la nueva temporada
new_data_path = '../../data/partidos_2023-2024_limpio.csv'
new_data = pd.read_csv(new_data_path)
new_data['Score'] = new_data['Score'].str.replace('–', '-', regex=False)
cleaned_new_data = new_data.dropna(subset=['Score'])
cleaned_new_data['Home_goals'] = cleaned_new_data['Score'].str.extract('(\d+)-')[0].astype(int)
cleaned_new_data['Away_goals'] = cleaned_new_data['Score'].str.extract('-(\d+)')[0].astype(int)
cleaned_new_data['Result'] = (cleaned_new_data['Home_goals'] > cleaned_new_data['Away_goals']).astype(int) + \
                             (cleaned_new_data['Away_goals'] > cleaned_new_data['Home_goals']).astype(int) * 2

# Identificar y simular partidos no jugados
unplayed_matches = new_data[new_data['Score'].isna()]
simulated_results = []
np.random.seed(42)
for _, row in unplayed_matches.iterrows():
    result = np.random.choice([0, 1, 2], p=transition_matrix[1])
    simulated_results.append(result)
unplayed_matches = unplayed_matches.assign(Simulated_Result=simulated_results)

# Combinar resultados reales y simulados, y calcular puntos
full_results = pd.concat([
    cleaned_new_data[['Home', 'Away', 'Result']],
    unplayed_matches[['Home', 'Away', 'Simulated_Result']].rename(columns={'Simulated_Result': 'Result'})
])

def calculate_points(df):
    points = {}
    for _, row in df.iterrows():
        home, away, result = row['Home'], row['Away'], row['Result']
        if home not in points:
            points[home] = 0
        if away not in points:
            points[away] = 0
        if result == 1:
            points[home] += 3
        elif result == 2:
            points[away] += 3
        elif result == 0:
            points[home] += 1
            points[away] += 1
    return points

points = calculate_points(full_results)
sorted_teams = sorted(points.items(), key=lambda x: x[1], reverse=True)
sorted_teams[:10]

C:\Users\albab\AppData\Local\Temp\ipykernel_35728\4096186532.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_new_data['Home_goals'] = cleaned_new_data['Score'].str.extract('(\d+)-')[0].astype(int)
C:\Users\albab\AppData\Local\Temp\ipykernel_35728\4096186532.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_new_data['Away_goals'] = cleaned_new_data['Score'].str.extract('-(\d+)')[0].astype(int)
C:\Users\albab\AppData\Local\Temp\ipykernel_35728\4096186532.py:8: SettingWithCopyWarning: 
A 

[('Paris S-G', 12),
 ('Bayern Munich', 10),
 ('Dortmund', 10),
 ('Real Madrid', 9),
 ('Manchester City', 8),
 ('Barcelona', 7),
 ('Atlético Madrid', 6),
 ('Arsenal', 4),
 ('Lazio', 3),
 ('Inter', 3)]